In [1]:
# Import statements
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import acf

In [20]:
# 10 year age classes
def map_to_age_class(age):
    age_map = {
        '00_04': '00_09', '05_09': '00_09',
        '10_14': '10_19', '15_19': '10_19',
        '20_24': '20_29', '25_29': '20_29',
        '30_34': '30_39', '35_39': '30_39',
        '40_44': '40_49', '45_49': '40_49',
        '50_54': '50_59', '55_59': '50_59',
        '60_64': '60_69', '65_69': '60_69',
        '70_74': '70_79', '75_79': '70_79',
        '80_84': '80_89', '85_89': '80_89',
        '90+': '90+'
        # Add additional mappings as needed
    }
    return age_map.get(age, age)

pop_full = pd.read_csv('ONS-population_2021-08-05.csv')
pop_full = (pop_full[(~pop_full['age'].isin(['00_59', '60+', 'unassigned','18_64','ALL','65_84','6_17'])) & (pop_full['category']=='AGE_ONLY')]).drop(columns=['category','gender'])
pop_full = pop_full[pop_full['areaCode'].str.startswith('E')]
pop_full.rename(columns={'areaCode':'area_code'},inplace=True)
pop_full['age_class'] = pop_full['age'].apply(map_to_age_class)
pop_full.drop(columns='age',inplace=True)
pop_full = pop_full.groupby(['area_code','age_class']).sum().reset_index()

cases = pd.read_csv('ltla_2023-12-14 (1).csv')
cases = (cases[~cases['age'].isin(['00_59', '60+', 'unassigned'])]).drop(columns=['areaType','rollingSum','rollingRate'])
cases['date'] = pd.to_datetime(cases['date'])
cases = cases[cases['date'] <= pd.to_datetime('2021-12-31')]
cases['age_class'] = cases['age'].apply(map_to_age_class)
cases.set_index('date',inplace=True)
cases.drop(columns='age',inplace=True)
cases = cases.groupby(['areaCode','areaName','age_class','date']).sum().reset_index().set_index('date')
cases = cases.rename({'areaCode':'area_code','areaName':'area_name'},axis=1)
weekly_counts_total = cases.groupby(['area_code','area_name','age_class']).rolling(7).sum().reset_index()
weekly_counts_total

,area_code,area_name,age_class,date,cases
0,E06000001,Hartlepool,00_09,2020-03-16,NaN
1,E06000001,Hartlepool,00_09,2020-03-17,NaN
2,E06000001,Hartlepool,00_09,2020-03-18,NaN
3,E06000001,Hartlepool,00_09,2020-03-19,NaN
4,E06000001,Hartlepool,00_09,2020-03-20,NaN
...,...,...,...,...,...
2097855,E09000033,Westminster,90+,2021-12-27,15.0
2097856,E09000033,Westminster,90+,2021-12-28,16.0
2097857,E09000033,Westminster,90+,2021-12-29,16.0
2097858,E09000033,Westminster,90+,2021-12-30,11.0


In [21]:
# Convert incidence to a proportion to allow for cross-LTLA comparison
weekly_counts_total = pd.merge(weekly_counts_total,pop_full,on=['area_code','age_class'])
weekly_counts_total['incidence'] = weekly_counts_total['cases'] / weekly_counts_total['population']

npis = pd.read_csv('England_LTLA_NHSER.csv')
npis = npis[['NHSER_name','LTLA_name']]
npis.rename(columns={'NHSER_name':'region','LTLA_name':'ltla20nm'},inplace=True)
npis.drop_duplicates(inplace=True)
manual_dat = pd.DataFrame({'ltla20nm': ['Wycombe','South Bucks','Chiltern','Aylesbury Vale','Cornwall and Isles of Scilly', 'Hackney and City of London'], \
                           'region': ['South East', 'South East', 'South East', 'South East', 'South West', 'London']})
npis = pd.concat([npis,manual_dat])

cases_region = pd.merge(cases.reset_index(), npis, left_on='area_name', right_on='ltla20nm')
cases_region.drop(columns='ltla20nm',inplace=True)

weekly_counts_total = pd.merge(weekly_counts_total, npis, left_on='area_name', right_on='ltla20nm')
weekly_counts_total.drop(columns='ltla20nm',inplace=True)

regional_mean_total = weekly_counts_total.groupby(['region', 'date', 'age_class'])['incidence'].mean().reset_index()
regional_mean_total.rename(columns={'incidence': 'regional_mean_incidence'}, inplace=True)
regional_mean_total

,region,date,age_class,regional_mean_incidence
0,East of England,2020-02-03,00_09,NaN
1,East of England,2020-02-03,10_19,NaN
2,East of England,2020-02-03,20_29,NaN
3,East of England,2020-02-03,30_39,NaN
4,East of England,2020-02-03,40_49,NaN
...,...,...,...,...
48815,South West,2021-12-31,50_59,0.011950
48816,South West,2021-12-31,60_69,0.007921
48817,South West,2021-12-31,70_79,0.005142
48818,South West,2021-12-31,80_89,0.003847


In [22]:
weekly_counts_total = pd.merge(weekly_counts_total, regional_mean_total, on=['region', 'date', 'age_class'])
weekly_counts_total['detrended'] = weekly_counts_total['incidence'] - weekly_counts_total['regional_mean_incidence']
weekly_counts_total

,area_code,area_name,age_class,date,cases,population,incidence,region,regional_mean_incidence,detrended
0,E06000001,Hartlepool,00_09,2020-03-16,NaN,10921,NaN,North East and Yorkshire,6.097003e-07,NaN
1,E06000001,Hartlepool,00_09,2020-03-17,NaN,10921,NaN,North East and Yorkshire,0.000000e+00,NaN
2,E06000001,Hartlepool,00_09,2020-03-18,NaN,10921,NaN,North East and Yorkshire,2.887278e-06,NaN
3,E06000001,Hartlepool,00_09,2020-03-19,NaN,10921,NaN,North East and Yorkshire,2.616595e-06,NaN
4,E06000001,Hartlepool,00_09,2020-03-20,NaN,10921,NaN,North East and Yorkshire,2.995047e-06,NaN
...,...,...,...,...,...,...,...,...,...,...
2097855,E09000033,Westminster,90+,2021-12-27,15.0,1863,0.008052,London,8.779400e-03,-0.000728
2097856,E09000033,Westminster,90+,2021-12-28,16.0,1863,0.008588,London,1.016454e-02,-0.001576
2097857,E09000033,Westminster,90+,2021-12-29,16.0,1863,0.008588,London,1.148086e-02,-0.002893
2097858,E09000033,Westminster,90+,2021-12-30,11.0,1863,0.005904,London,1.303980e-02,-0.007135


In [32]:
weekly_counts_total = pd.merge(weekly_counts_total, regional_mean_total, on=['region', 'date', 'age_class'])
weekly_counts_total['detrended'] = weekly_counts_total['incidence'] - weekly_counts_total['regional_mean_incidence']
weekly_counts_total

,area_code,area_name,age_class,date,cases,population,incidence,region,regional_mean_incidence,detrended
0,E06000001,Hartlepool,00_09,2021-01-01,NaN,10921,NaN,North East and Yorkshire,NaN,NaN
1,E06000001,Hartlepool,00_09,2021-01-02,NaN,10921,NaN,North East and Yorkshire,NaN,NaN
2,E06000001,Hartlepool,00_09,2021-01-03,NaN,10921,NaN,North East and Yorkshire,NaN,NaN
3,E06000001,Hartlepool,00_09,2021-01-04,NaN,10921,NaN,North East and Yorkshire,NaN,NaN
4,E06000001,Hartlepool,00_09,2021-01-05,NaN,10921,NaN,North East and Yorkshire,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1149745,E09000033,Westminster,90+,2021-12-27,15.0,1863,0.008052,London,0.008779,-0.000728
1149746,E09000033,Westminster,90+,2021-12-28,16.0,1863,0.008588,London,0.010165,-0.001576
1149747,E09000033,Westminster,90+,2021-12-29,16.0,1863,0.008588,London,0.011481,-0.002893
1149748,E09000033,Westminster,90+,2021-12-30,11.0,1863,0.005904,London,0.013040,-0.007135


In [23]:
wide_data_total = weekly_counts_total.pivot(index='date', columns=['area_name','age_class'], values='detrended')
wide_data_incidence = weekly_counts_total.pivot(index='date', columns=['area_name','age_class'], values='incidence')

wide_data_incidence.to_csv('wide_incidence_10year_full.csv')
wide_data_total.to_csv('wide_total_10year_full.csv')
weekly_counts_total.to_csv('weekly_total_10year_full.csv')

In [32]:
pop_full = pd.read_csv('ONS-population_2021-08-05.csv')
pop_full = (pop_full[(~pop_full['age'].isin(['00_59', '60+', 'unassigned','18_64','ALL','65_84','6_17'])) & (pop_full['category']=='AGE_ONLY')]).drop(columns=['category','gender'])
pop_full = pop_full[pop_full['areaCode'].str.startswith('E')]
pop_full.rename(columns={'areaCode':'area_code'},inplace=True)


cases = pd.read_csv('ltla_2023-12-14 (1).csv')
cases = (cases[~cases['age'].isin(['00_59', '60+', 'unassigned'])]).drop(columns=['areaType','rollingSum','rollingRate'])
cases['date'] = pd.to_datetime(cases['date'])
cases = cases[cases['date'] <= pd.to_datetime('2021-12-31')]
cases.set_index('date',inplace=True)
cases.sort_index(inplace=True)
cases = cases.rename({'areaCode':'area_code','areaName':'area_name'},axis=1)
weekly_counts_total = cases.groupby(['area_code','area_name','age']).rolling(7).sum().reset_index()
weekly_counts_total

,area_code,area_name,age,date,cases
0,E06000001,Hartlepool,00_04,2020-03-16,NaN
1,E06000001,Hartlepool,00_04,2020-03-17,NaN
2,E06000001,Hartlepool,00_04,2020-03-18,NaN
3,E06000001,Hartlepool,00_04,2020-03-19,NaN
4,E06000001,Hartlepool,00_04,2020-03-20,NaN
...,...,...,...,...,...
3985929,E09000033,Westminster,90+,2021-12-27,15.0
3985930,E09000033,Westminster,90+,2021-12-28,16.0
3985931,E09000033,Westminster,90+,2021-12-29,16.0
3985932,E09000033,Westminster,90+,2021-12-30,11.0


In [33]:
# Convert incidence to a proportion to allow for cross-LTLA comparison
weekly_counts_total = pd.merge(weekly_counts_total,pop_full,on=['area_code','age'])
weekly_counts_total['incidence'] = weekly_counts_total['cases'] / weekly_counts_total['population']

npis = pd.read_csv('England_LTLA_NHSER.csv')
npis = npis[['NHSER_name','LTLA_name']]
npis.rename(columns={'NHSER_name':'region','LTLA_name':'ltla20nm'},inplace=True)
npis.drop_duplicates(inplace=True)
manual_dat = pd.DataFrame({'ltla20nm': ['Wycombe','South Bucks','Chiltern','Aylesbury Vale','Cornwall and Isles of Scilly', 'Hackney and City of London'], \
                           'region': ['South East', 'South East', 'South East', 'South East', 'South West', 'London']})
npis = pd.concat([npis,manual_dat])

cases_region = pd.merge(cases.reset_index(), npis, left_on='area_name', right_on='ltla20nm')
cases_region.drop(columns='ltla20nm',inplace=True)

weekly_counts_total = pd.merge(weekly_counts_total, npis, left_on='area_name', right_on='ltla20nm')
weekly_counts_total.drop(columns='ltla20nm',inplace=True)

regional_mean_total = weekly_counts_total.groupby(['region', 'date', 'age'])['incidence'].mean().reset_index()
regional_mean_total.rename(columns={'incidence': 'regional_mean_incidence'}, inplace=True)
regional_mean_total

,region,date,age,regional_mean_incidence
0,East of England,2020-02-03,00_04,NaN
1,East of England,2020-02-03,05_09,NaN
2,East of England,2020-02-03,10_14,NaN
3,East of England,2020-02-03,15_19,NaN
4,East of England,2020-02-03,20_24,NaN
...,...,...,...,...
92753,South West,2021-12-31,70_74,0.005350
92754,South West,2021-12-31,75_79,0.004852
92755,South West,2021-12-31,80_84,0.003891
92756,South West,2021-12-31,85_89,0.003783


In [34]:
weekly_counts_total = pd.merge(weekly_counts_total, regional_mean_total, on=['region', 'date', 'age'])
weekly_counts_total['detrended'] = weekly_counts_total['incidence'] - weekly_counts_total['regional_mean_incidence']
weekly_counts_total

,area_code,area_name,age,date,cases,population,incidence,region,regional_mean_incidence,detrended
0,E06000001,Hartlepool,00_04,2020-03-16,NaN,5147,NaN,North East and Yorkshire,0.000000,NaN
1,E06000001,Hartlepool,00_04,2020-03-17,NaN,5147,NaN,North East and Yorkshire,0.000000,NaN
2,E06000001,Hartlepool,00_04,2020-03-18,NaN,5147,NaN,North East and Yorkshire,0.000006,NaN
3,E06000001,Hartlepool,00_04,2020-03-19,NaN,5147,NaN,North East and Yorkshire,0.000006,NaN
4,E06000001,Hartlepool,00_04,2020-03-20,NaN,5147,NaN,North East and Yorkshire,0.000005,NaN
...,...,...,...,...,...,...,...,...,...,...
3985929,E09000033,Westminster,90+,2021-12-27,15.0,1863,0.008052,London,0.008779,-0.000728
3985930,E09000033,Westminster,90+,2021-12-28,16.0,1863,0.008588,London,0.010165,-0.001576
3985931,E09000033,Westminster,90+,2021-12-29,16.0,1863,0.008588,London,0.011481,-0.002893
3985932,E09000033,Westminster,90+,2021-12-30,11.0,1863,0.005904,London,0.013040,-0.007135


In [35]:
wide_data_total = weekly_counts_total.pivot(index='date', columns=['area_name','age'], values='detrended')
wide_data_incidence = weekly_counts_total.pivot(index='date', columns=['area_name','age'], values='incidence')

wide_data_incidence.to_csv('wide_incidence_full.csv')
wide_data_total.to_csv('wide_total_full.csv')
npis.to_csv('ltla_to_nhs.csv')
weekly_counts_total.to_csv('weekly_total_full.csv')